# Using the simulator

In this notebook you will learn how to use the simulator to test out methods.

## Setting up a connection with the robot

As described in the [basic liquid handling tutorial](basic), we will use the {class}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler` class to control the robot. This time, however, instead of using the Hamilton {class}`~pylabrobot.liquid_handling.backends.hamilton.STAR.STAR` backend, we are using virtual the {class}`~pylabrobot.liquid_handling.backends.simulation.simulator_backend.SimulatorBackend` backend. This means that liquid handling will work exactly the same, but the commands are sent to the simulator instead of a real physical robot.

In [67]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends.simulation.simulator_backend import SimulatorBackend

In [69]:
## For Hamilton:
# from pylabrobot.resources.hamilton import STARLetDeck

## For Opentrons:
from pylabrobot.resources.opentrons import deck

In [76]:
## For Hamilton:
# sb = SimulatorBackend(open_browser=False)
# lh = LiquidHandler(backend=sb, deck=STARLetDeck())

## For Opentrons: OPS OPS THIS DOES NOT WORK!!!!!!
sb = SimulatorBackend(ws_host="127.0.0.1", fs_host="127.0.0.1")
lh = LiquidHandler(backend=sb, deck=deck)

Calling {func}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler.setup` will set up the simulation server and open it in a new browser tab.

In [77]:
await lh.setup()

Exception in thread Thread-24 (callback):
Traceback (most recent call last):
  File "C:\Users\vikmol\AppData\Local\Programs\Python\Python310\Lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\vikmol\AppData\Local\Programs\Python\Python310\Lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "c:\users\vikmol\onedrive\dokumenter\github\pylabrobot_dalsa\pylabrobot\liquid_handling\liquid_handler.py", line 152, in callback
    loop.run_until_complete(func(*args, **kwargs))
  File "C:\Users\vikmol\AppData\Local\Programs\Python\Python310\Lib\asyncio\base_events.py", line 649, in run_until_complete
    return future.result()
  File "c:\users\vikmol\onedrive\dokumenter\github\pylabrobot_dalsa\pylabrobot\liquid_handling\backends\websocket.py", line 167, in assigned_resource_callback
    "resource": resource.serialize(),
AttributeError: module 'pylabrobot.resources.opentrons.deck' has no attribute 'serialize'


Websocket server started at http://127.0.0.1:2125
File server started at http://127.0.0.1:1337 . Open this URL in your browser.


AttributeError: module 'pylabrobot.resources.opentrons.deck' has no attribute 'children'

For the optimal experience, we recommend that you run the notebook and simulator side by side.

In [78]:
sb.wait_for_connection()

![The empty simulator](./img/simulator/empty.png)

## Assigning plates and tips

With the simulator, {func}`assigning resources <pylabrobot.resources.Deck.assign_child_resource>` has the additional affect of placing the resources on the simulated deck. They will appear immediately.

In [39]:
## For Hamilton:
# from pylabrobot.resources import (
#     TIP_CAR_480_A00,
#     PLT_CAR_L5AC_A00,
#     Cos_96_DW_1mL,
#     HTF_L
# )

## For Opentrons:
from pylabrobot.resources.opentrons import (
  opentrons_96_filtertiprack_20ul
)
from pylabrobot.resources.corning_costar import (
  Cos_96_EZWash
)


In [42]:
## For Hamilton:
# tip_car = TIP_CAR_480_A00(name='tip carrier')
# tip_car[0] = tips = HTF_L(name='tips_01')
# tip_car[1] = HTF_L(name='tips_02')
# tip_car[2] = HTF_L(name='tips_03')
# tip_car[3] = HTF_L(name='tips_04')
# tip_car[4] = HTF_L(name='tips_05')

## For Opentrons:
tips = opentrons_96_filtertiprack_20ul(name="tip_rack")
plate = Cos_96_EZWash(name="plate")


In [43]:
## For Hamilton:
# lh.deck.assign_child_resource(tip_car, rails=15)

## For Opentrons:
lh.deck.assign_child_resource(tips, slot=1)
lh.deck.assign_child_resource(plate, slot=2)


TypeError: HamiltonDeck.assign_child_resource() got an unexpected keyword argument 'slot'

In [12]:
plt_car = PLT_CAR_L5AC_A00(name='plate carrier')
plt_car[0] = plate_1 = Cos_96_DW_1mL(name='plate_01')
plt_car[1] = plate_2 = Cos_96_DW_1mL(name='plate_02')
plt_car[2] = plate_3 = Cos_96_DW_1mL(name='plate_03')

In [13]:
lh.deck.assign_child_resource(plt_car, rails=8)

![The simulator after the resources have been assigned](./img/simulator/assignment.png)

## Build the deck layout: placing resources

Where you would manually place the resources like tips and liquid on the deck when using a physical system, with the Simulator you can add them using code.

### Tips

Let's use {func}`~pylabrobot.liquid_handling.backends.simulation.SimulatorBackend.fill_tip_rack` to place tips at all spots in the tip rack in location `0`.

In [14]:
tiprack = lh.get_resource("tips_01")
tiprack

TipRack(name=tips_01, size_x=122.4, size_y=82.6, size_z=20.0, location=(000.000, 000.000, 000.000))

In [15]:
await sb.fill_tip_rack(tiprack)


You can precisely control the presence of tips using {func}`~pylabrobot.liquid_handling.backends.simulation.SimulatorBackend.edit_tips`.

In [16]:
tips4 = lh.get_resource("tips_04")
await sb.edit_tips(tips4, pattern=[[True]*6 + [False]*6]*8)

In [17]:
await sb.edit_tips(lh.get_resource("tips_03"), pattern=[[True, False]*6]*8)

In [18]:
await sb.edit_tips(lh.get_resource("tips_02"), pattern=[[True, True, False, False]*3]*8)

### Liquids

Adding liquid to wells works similarly. You can use {func}`~pylabrobot.liquid_handling.backends.simulation.SimulatorBackend.adjust_well_volume` to adjust the volume of individual wells in each resource. Note that the opacity of the well matches the volume of the well.

In [19]:
plate_1_liquids = [[(None, 500)]]*96
await sb.adjust_wells_liquids(plate_1, liquids=plate_1_liquids)

In [20]:
plate_2_liquids = [[(None, 100)], [(None, 500)]]*(96//2)
await sb.adjust_wells_liquids(plate_2, liquids=plate_2_liquids)

Using the simulator backend we have adjusted the volume in the simulator, which you can best compare to adding liquid in reality. Now we need to update the wells (that live in Python) to reflect how much volume is in them, so that LiquidHandler can validate your actions. This is done using {func}`~pylabrobot.resources.Plate.set_well_liquids`. Note that this can be done in all liquid handling protocols, not just the simulator.

In [22]:
plate_1.set_well_liquids(plate_1_liquids)
plate_2.set_well_liquids(plate_2_liquids)

![Simulator after the tips have been placed and the volumes adjusted](./img/simulator/resources.png)

## Liquid handling

Once the layout is complete, you can run the same commands as described in the [basic liquid handling tutorial](basic).

### Picking up tips

In [23]:
tip_0 = lh.get_resource("tips_01")

In [24]:
await lh.pick_up_tips(tip_0["A1", "B2", "C3", "D4"])

In [25]:
await lh.drop_tips(tip_0["A1", "B2", "C3", "D4"])

### Aspirating and dispensing

In [26]:
await lh.pick_up_tips(tip_0["A1"])

In [27]:
plate = lh.get_resource("plate_01")

In [28]:
await lh.aspirate(plate["A2"], vols=[300])

In [29]:
await lh.dispense(plate_2["A1"], vols=[300])

In [30]:
await lh.drop_tips(tip_0["A1"])

### Aspirating using CoRe 96

The CoRe 96 head supports liquid handling operations for 96 channels at once. Here's how to use:

- {func}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler.pick_up_tips96` for picking up 96 tips;
- {func}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler.aspirate_plate` for aspirating liquid from an entire plate at once;
- {func}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler.dispense_plate` for dispensing liquid to an entire plate at once;
- {func}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler.drop_tips96` for dropping tips to the tip rack.


In [31]:
await lh.pick_up_tips96(tiprack)

In [32]:
await lh.aspirate_plate(plt_car[0].resource, volume=200)

In [33]:
await lh.dispense_plate(plt_car[2].resource, volume=200)

In [34]:
await lh.drop_tips96(tiprack)

![The simulator after the liquid handling operations completed](./img/simulator/after_lh.png)

## Shutting down

When you're done, remember to shut down the simulator by calling {func}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler.stop`.

In [35]:
await lh.stop()

connection handler failed
Traceback (most recent call last):
  File "c:\Users\vikmol\OneDrive\Dokumenter\GitHub\pylabrobot_DALSA\venv\lib\site-packages\websockets\legacy\server.py", line 236, in handler
    await self.ws_handler(self)
  File "c:\users\vikmol\onedrive\dokumenter\github\pylabrobot_dalsa\pylabrobot\liquid_handling\backends\websocket.py", line 138, in _socket_handler
    await self.handle_event(data.get("event"), data)
  File "c:\users\vikmol\onedrive\dokumenter\github\pylabrobot_dalsa\pylabrobot\liquid_handling\backends\websocket.py", line 112, in handle_event
    await self.websocket.send(json.dumps({"event": "pong"}))
  File "c:\users\vikmol\onedrive\dokumenter\github\pylabrobot_dalsa\pylabrobot\liquid_handling\backends\websocket.py", line 69, in websocket
    raise RuntimeError("No websocket connection has been established.")
RuntimeError: No websocket connection has been established.
